In [10]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

print(sys.path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['/mnt/hdd2/transformer-entropy-ids/notebooks', '/home/tiendat/miniconda3/envs/piptorchtf/lib/python39.zip', '/home/tiendat/miniconda3/envs/piptorchtf/lib/python3.9', '/home/tiendat/miniconda3/envs/piptorchtf/lib/python3.9/lib-dynload', '', '/home/tiendat/miniconda3/envs/piptorchtf/lib/python3.9/site-packages', '../', '../']


In [12]:
import torch
import torch.nn as nn
import math
from dataPreprocess import DatasetPreprocess
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, classification_report, confusion_matrix
import os
import time
import warnings
import torch.nn.functional as F
import time

In [13]:
def draw_confusion(label_y, pre_y, path):
    confusion = confusion_matrix(label_y, pre_y)
    print(confusion)

def write_result(label_y, pre_y, classes_num):
    if classes_num > 2:
        accuracy = accuracy_score(label_y, pre_y)
        macro_precision = precision_score(label_y, pre_y, average='macro')
        macro_recall = recall_score(label_y, pre_y, average='macro')
        macro_f1 = f1_score(label_y, pre_y, average='macro')
        micro_precision = precision_score(label_y, pre_y, average='micro')
        micro_recall = recall_score(label_y, pre_y, average='micro')
        micro_f1 = f1_score(label_y, pre_y, average='micro')
        print('  -- test result: ')
        print('    -- accuracy: ', accuracy)
        print('    -- macro precision: ', macro_precision)
        print('    -- macro recall: ', macro_recall)
        print('    -- macro f1 score: ', macro_f1)
        print('    -- micro precision: ', micro_precision)
        print('    -- micro recall: ', micro_recall)
        print('    -- micro f1 score: ', micro_f1)
        report = classification_report(label_y, pre_y)
        print(report)
    else:
        accuracy = accuracy_score(label_y, pre_y)
        precision = precision_score(label_y, pre_y)
        recall = recall_score(label_y, pre_y)
        f1 = f1_score(label_y, pre_y)
        print('  -- test result: ')
        print('    -- accuracy: ', accuracy)
        print('    -- recall: ', recall)
        print('    -- precision: ', precision)
        print('    -- f1 score: ', f1)
        report = classification_report(label_y, pre_y)
        print(report)

In [14]:
class Config:
    def __init__(self):
        self.model_name = 'Transformer'
        self.slide_window = 1
        self.slsum_count = 8 #int(math.pow(16, self.slide_window))  # 滑动窗口计数的特征的长度 n-gram?
        self.dnn_out_d = 8 # 经过DNN后的滑动窗口计数特征的维度 Dimensions of sliding window count features after DNN 8
        self.head_dnn_out_d = 32 
        self.d_model = self.dnn_out_d + self.head_dnn_out_d # transformer的输入的特征的维度, dnn_out_d + 包头长度 The dimension of the input feature of the transformer, dnn_out_d + header length
        self.pad_size = 29
        self.window_size = 29
        self.max_time_position = 10000
        self.nhead = 5 # ori: 5
        self.num_layers = 5
        self.gran = 1e-6
        self.log_e = 2
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.classes_num = 5 
        self.batch_size = 10
        self.epoch_num = 20
        self.lr = 0.00001 #0.00001 learning rate 
        self.train_pro = 0.7  # 训练集比例 Ratio of training set

        self.root_dir = '../data/Processed/TFRecord_w29_s29/1/'
        self.model_save_path = '../model/' + self.model_name + '/'
        if not os.path.exists(self.model_save_path):
            os.mkdir(self.model_save_path)
        self.result_file = '/mnt/hdd2/transformer-entropy-ids/result/trans8_performance.txt'

        self.isload_model = False  # 是否加载模型继续训练 Whether to load the model and continue training
        self.start_epoch = 18  # 加载的模型的epoch The epoch of the loaded model
        # self.model_path = '../model/' + self.model_name + '/' + self.model_name + '_model_' + str(self.start_epoch) + '.pth'  # 要使用的模型的路径 path to the model to use
        self.model_path = '../model/Transformer/best_model.pth'
        
class DNN(nn.Module):
    def __init__(self, d_in, d_out):  # config.slsum_count, config.dnn_out_d
        super(DNN, self).__init__()
        self.l1 = nn.Linear(d_in, 128)
        self.l2 = nn.Linear(128, 64)
        self.l3 = nn.Linear(64, d_out)

    def forward(self, x):
        # print('x: ', x.numpy()[0])
        out = F.relu(self.l1(x))
        out = F.relu(self.l2(out))
        out = F.relu(self.l3(out))
        # print('dnn out: ', out.detach().numpy()[0])
        return out

class Time_Positional_Encoding(nn.Module):
    def __init__(self, embed, max_time_position, device):
        super(Time_Positional_Encoding, self).__init__()
        self.device = device

    def forward(self, x, time_position):
        out = x.permute(1, 0, 2)
        out = out + nn.Parameter(time_position, requires_grad=False).to(self.device)
        out = out.permute(1, 0, 2)
        return out

class MyTrans(nn.Module):
    def __init__(self, config):
        super(MyTrans, self).__init__()
        self.dnn = DNN(config.slsum_count, config.dnn_out_d).to(config.device)
        self.head_dnn = DNN(4, config.head_dnn_out_d).to(config.device)
        self.position_embedding = Time_Positional_Encoding(config.d_model, config.max_time_position, config.device).to(config.device)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=config.d_model, nhead=config.nhead).to(config.device)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=config.num_layers).to(config.device)
        self.fc = nn.Linear(config.d_model, config.classes_num).to(config.device)
        self.pad_size = config.pad_size
        self.dnn_out_d = config.dnn_out_d
        self.head_dnn_out_d = config.head_dnn_out_d

    def forward(self, header, sl_sum, mask, time_position):
        dnn_out = torch.empty((sl_sum.shape[0], self.dnn_out_d, 0)).to(config.device)

        for i in range(self.pad_size):
            tmp = self.dnn(sl_sum[:, i, :]).unsqueeze(2)
            dnn_out = torch.concat((dnn_out, tmp), dim=2)
        dnn_out = dnn_out.permute(0, 2, 1)

        head_dnn_out = torch.empty((header.shape[0], self.head_dnn_out_d, 0)).to(config.device)
        for i in range(self.pad_size):
            tmp = self.head_dnn(header[:, i, :]).unsqueeze(2)
            head_dnn_out = torch.concat((head_dnn_out, tmp), dim=2)
        head_dnn_out = head_dnn_out.permute(0, 2, 1)

        x = torch.concat((head_dnn_out, dnn_out), dim=2).permute(1, 0, 2)

        out = self.position_embedding(x, time_position)
        out = self.transformer_encoder(out, src_key_padding_mask=mask)
        out = out.permute(1, 0, 2)
        out = torch.sum(out, 1)
        out = self.fc(out)
        return out


In [15]:
config = Config()
print(config.device)
print(config.model_path)

seed = 1
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

test_dataset = DatasetPreprocess(config.root_dir, config.window_size, config.pad_size, config.d_model, config.max_time_position, config.gran, config.log_e, is_train=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=config.batch_size)
print('finish load data')

model = torch.load(config.model_path)
print("Model loaded")

loss_func = nn.CrossEntropyLoss().to(config.device)
opt = torch.optim.Adam(model.parameters(), lr=config.lr)

cuda
../model/Transformer/Transformer_model_18.pth
finish load data
Model loaded


In [20]:
label_y = []
pre_y = []
timeCalc = [] 
with torch.no_grad():
    for j, test_sample_batch in enumerate(test_loader):
        start_time = time.time()
        test_header = test_sample_batch['header'].type(torch.FloatTensor).to(config.device)
        test_sl_sum = test_sample_batch['payload'].type(torch.FloatTensor).to(config.device)
        test_mask = test_sample_batch['mask'].to(config.device)
        test_label = test_sample_batch['label'].to(config.device)
        test_time_position = test_sample_batch['time'].to(config.device)
            
        
        test_out = model(test_header, test_sl_sum, test_mask, test_time_position)

        pre = torch.max(test_out, 1)[1].cpu().numpy()
        pre_y = np.concatenate([pre_y, pre], 0)
        label_y = np.concatenate([label_y, test_label.cpu().numpy()], 0)
        end_time = time.time()
        execution_time = end_time - start_time
        timeCalc.append(execution_time)
    write_result(label_y, pre_y, config.classes_num)
    draw_confusion(label_y, pre_y, '')

  -- test result: 
    -- accuracy:  1.0
    -- macro precision:  1.0
    -- macro recall:  1.0
    -- macro f1 score:  1.0
    -- micro precision:  1.0
    -- micro recall:  1.0
    -- micro f1 score:  1.0
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    105833
         1.0       1.00      1.00      1.00     11236
         2.0       1.00      1.00      1.00     13346
         3.0       1.00      1.00      1.00     19585
         4.0       1.00      1.00      1.00     21412

    accuracy                           1.00    171412
   macro avg       1.00      1.00      1.00    171412
weighted avg       1.00      1.00      1.00    171412

[[105833      0      0      0      0]
 [     0  11236      0      0      0]
 [     0      0  13346      0      0]
 [     0      0      0  19585      0]
 [     0      0      0      0  21412]]


In [19]:
def calculate_average(arr):
    if not arr:
        raise ValueError("Input array is empty. Cannot calculate the average.")
    
    total = sum(arr)
    average = total / len(arr)
    return average
print("Execution time:", calculate_average(timeCalc), "seconds")

Execution time: 0.012494143687528513 seconds


In [8]:
import time

start_time = time.time()
for i in range(20):
    for j in range(20):
        a = i + j
end_time = time.time()

print(end_time-start_time)

0.0001251697540283203
